In [23]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report

## TFIDF

In [24]:
train_data = pd.read_pickle(open("tfidf/train_tfidf.pkl", "rb"))
train_X = list(train_data['TFIDF'])
train_y = train_data['Sentiment']
dev_data = pd.read_pickle(open("tfidf/dev_tfidf.pkl", "rb"))
dev_X = list(dev_data['TFIDF'])
dev_y = dev_data['Sentiment']

In [25]:
gnb = GaussianNB()
gnb.fit(train_X, train_y)
labels_predict = gnb.predict(dev_X)
labels_predict

array(['negative', 'positive', 'negative', ..., 'positive', 'positive',
       'positive'], dtype='<U8')

In [26]:
print(classification_report(dev_y, labels_predict))
print(confusion_matrix(dev_y, labels_predict))

              precision    recall  f1-score   support

    negative       0.64      0.65      0.65      2000
    positive       0.65      0.64      0.64      2000

    accuracy                           0.65      4000
   macro avg       0.65      0.65      0.65      4000
weighted avg       0.65      0.65      0.65      4000

[[1301  699]
 [ 720 1280]]


## Embedding

In [27]:
train_data = pd.read_pickle(open("sentence-transformers/train_emb.pkl", "rb"))
train_X = list(train_data['TFIDF'])
train_y = train_data['Sentiment']
dev_data = pd.read_pickle(open("sentence-transformers/dev_emb.pkl", "rb"))
dev_X = list(dev_data['TFIDF'])
dev_y = dev_data['Sentiment']

In [28]:
gnb = GaussianNB()
gnb.fit(train_X, train_y)
labels_predict = gnb.predict(dev_X)
labels_predict

array(['negative', 'positive', 'negative', ..., 'positive', 'positive',
       'positive'], dtype='<U8')

In [29]:
print(classification_report(dev_y, labels_predict))
print(confusion_matrix(dev_y, labels_predict))

              precision    recall  f1-score   support

    negative       0.60      0.70      0.64      2000
    positive       0.64      0.53      0.58      2000

    accuracy                           0.61      4000
   macro avg       0.62      0.61      0.61      4000
weighted avg       0.62      0.61      0.61      4000

[[1398  602]
 [ 939 1061]]
